# Exercise 7: Discovering Tensorflow

In [58]:
# Load packages we need
import sys
import os
import datetime

import numpy as np
import sklearn

import scipy as sp
import pandas as pd

import tensorflow as tf

# we'll use keras for neural networks
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist

%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 20})

# Let's check our software versions
print('### Python version: ' + sys.version)
print('### Numpy version: ' + np.__version__)
print('### Scikit-learn version: ' + sklearn.__version__)
print('### Tensorflow version: ' + tf.__version__)
print('------------')


# load our packages / code
sys.path.insert(1, '../common/')
import utils
import plots

### Python version: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
### Numpy version: 1.18.5
### Scikit-learn version: 0.24.1
### Tensorflow version: 2.3.1
------------


In [2]:
# global parameters to control behavior of the pre-processing, ML, analysis, etc.

seed = 42 # deterministic seed
np.random.seed(seed) 
tf.random.set_seed(seed)

prop_vec = [24, 2, 2]

## How to think of Tensorflow? Is it like scikit-learn but for neural networks?

### Not really, think of Tensorflow as a kind of NumPy with additional features (i.e., ability to create computational graphs on tensors, automatically compute derivative, run operations on GPUs). (Tensorflow also has many high-level APIs.)

### What are tensors? Well formally they are multilinear maps from vector spaces to reals; but that doesn't matter the point is that tensors can represent scalars, vectors, matrices, etc.. 

### Beware that Tensorflow 2.0 is different from Tensorflow 1.0! In this course we'll use Tensorflow 2.0.

### Compared to TF 1.0:
### - TF 2.0 incorporates Keras as a high-level API
### - TF 2.0 does *eager* execution by default!
#### In TF 1.0 you would first build the computational graph (construction phase) and then you would execute it in a session (execution phase).

In [3]:
tf.executing_eagerly()

True

## How do we set the seed for Tensorflow?

In [4]:
tf.random.set_seed(seed)

## Let's get familiar with Tensorflow

In [5]:
scalar = 7 # a scalar in Python

scalar_tf = tf.constant(7) # a TF scalar

print(scalar)
print(scalar_tf)

7
tf.Tensor(7, shape=(), dtype=int32)


### Just like numpy array, tensors have a shape and dtype property

In [6]:
vector_np = np.array([3, -5, 9, 1])
print(vector_np)

vector_tf = tf.constant([3, -5, 9, 1])
print(vector_tf)

[ 3 -5  9  1]
tf.Tensor([ 3 -5  9  1], shape=(4,), dtype=int32)


### We can get the dtype, shape of tensor. We can also get at the underlying numpy array using numpy().

In [7]:
print('dtype: ' + str(vector_tf.dtype))
print('shape: ' + str(vector_tf.shape))

numpy_arr = vector_tf.numpy()
print('numpy array: {}, type: {}'.format(str(numpy_arr), type(numpy_arr)))

dtype: <dtype: 'int32'>
shape: (4,)
numpy array: [ 3 -5  9  1], type: <class 'numpy.ndarray'>


In [8]:
# we can also build a tensor out of a numpy array
matrix_np = np.array([[3, -7], [0, 9]])
matrix_tf = tf.constant(matrix_np)

print(matrix_tf)

tf.Tensor(
[[ 3 -7]
 [ 0  9]], shape=(2, 2), dtype=int64)


In [9]:
# We can construct tensors in similar ways to how we construct some numpy arrays. For example:

tf_ones = tf.ones((3,3))
print(tf_ones)
print()

# and

tf_unifrand = tf.random.uniform((2, 4))
print(tf_unifrand)
print()

tf_zeros_like_ones = tf.zeros_like(tf_ones)
print(tf_zeros_like_ones)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(3, 3), dtype=float32)

tf.Tensor(
[[0.6645621  0.44100678 0.3528825  0.46448255]
 [0.03366041 0.68467236 0.74011743 0.8724445 ]], shape=(2, 4), dtype=float32)

tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], shape=(3, 3), dtype=float32)


### We can check if something is a Tensor. For example:

In [10]:
print(tf.is_tensor(matrix_tf))

True


In [11]:
print(tf.is_tensor(matrix_tf.numpy()))

False


### We can also place tensors onto devices. For example:

In [12]:
with tf.device('/gpu:0'):
    matrix_on_gpu0 = tf.identity(matrix_tf) # won't work if you don't have a GPU
    
print(matrix_on_gpu0.device)

/job:localhost/replica:0/task:0/device:CPU:0


In [13]:
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num CPUs Available:  1
Num GPUs Available:  0


### We can do operations as follow

In [14]:
x = tf.constant([1, 3])
y = tf.constant([-1, 2])

add_x_y = tf.add(x, y)
print(add_x_y)

tf.Tensor([0 5], shape=(2,), dtype=int32)


In [15]:
# Can we do x + y?
x_plus_y = x + y
print(x_plus_y)

tf.Tensor([0 5], shape=(2,), dtype=int32)


In [16]:
# multiplication by a scalar
x_mult_mone = x * -1
print(x_mult_mone)

tf.Tensor([-1 -3], shape=(2,), dtype=int32)


In [17]:
# elementwise multiplication
x_mult_y = x * y
# or: x_mult_y = tf.multiply(x,y)
print(x_mult_y)

tf.Tensor([-1  6], shape=(2,), dtype=int32)


### what about matrix multiplication and similar ops?

In [18]:
A = tf.constant([[1, 0, 3], [0, -2, 5]])
B = tf.constant([2, -3])

print(A.shape)
print(B.shape)

A_transposed = tf.transpose(A)
print(A_transposed.shape)

B_reshaped = tf.reshape(B, (-1, 1))

print(B_reshaped.shape)

(2, 3)
(2,)
(3, 2)
(2, 1)


In [19]:
A_T_matrix_mult_B = tf.linalg.matmul(A_transposed, B_reshaped)
# or A_transposed @ B_reshaped

print(A_T_matrix_mult_B)

tf.Tensor(
[[ 2]
 [ 6]
 [-9]], shape=(3, 1), dtype=int32)


### Because tensors are immutable, we cannot change their values in place. This seems like it could be a problem because parameters of a model are variables whose values should change frequently.
### For this we can use: tf.Variable

#### We'll typically use those for model parameters and other variables that need to change often in place.

In [20]:
# Let's declare a variable
# variables in TF represent tensors and you change their values by running operations (ops) on them
x = tf.Variable([7, 3], name="x")   # we can name variables (we don't have to, but we can)

In [21]:
print(x)

<tf.Variable 'x:0' shape=(2,) dtype=int32, numpy=array([7, 3], dtype=int32)>


In [22]:
# Variables also have shape and dtype, etc.
print(x.shape, x.dtype, x.name)

(2,) <dtype: 'int32'> x:0


In [23]:
# if you do ops on a variable the result is a tensor not a variable!
xsquared = tf.square(x)
print(xsquared)

tf.Tensor([49  9], shape=(2,), dtype=int32)


In [24]:
# but variables unlike constant can have their values changed in-place (e.g., using one of the assign*() methods). 
# For example:
x.assign(tf.constant([-1, 0]))
print(x)

x.assign_add(tf.constant([3, 3]))
print(x)

<tf.Variable 'x:0' shape=(2,) dtype=int32, numpy=array([-1,  0], dtype=int32)>
<tf.Variable 'x:0' shape=(2,) dtype=int32, numpy=array([2, 3], dtype=int32)>


In [25]:
# However, shapes must be compatible!
x.assign(tf.constant([5, 9, -17]))

ValueError: Shapes (2,) and (3,) are incompatible

## Cool (and important) feature: automatic differentiation

In [26]:
x = tf.Variable(2, name="x")

### Suppose we want to compute the derivative of x ** 3. Clearly it's 3 x ** 2
### We can do it using tf.GradientTape to keep track of the operations on tensor and then compute the gradient afterwards

In [27]:
# Note: to watch a tensor it must be floating point, so we'll cast x
x = tf.cast(x, dtype=tf.float16)

with tf.GradientTape() as tape:
    tape.watch(x) # we tell the tape to watch variable 'x'
    # now we can do operations like x ** 3
    y = x ** 3
    
    
## What is y?
print(y)

tf.Tensor(8.0, shape=(), dtype=float16)


In [28]:
## What is the gradient of y wrt x?
# we want the gradient of y (x**3) with respect to x
grad_xcube = tape.gradient(target=y, sources=x)

In [29]:
print(grad_xcube)

tf.Tensor(12.0, shape=(), dtype=float16)


In [30]:
print((3 * x**2).numpy())

12.0


### Note: once we get the gradients from the tape, the resources are released.

In [31]:
# This will cause an error
grad_xcube2 = tape.gradient(target=y, sources=x)

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

### But we can create a persistent tape if we want. For example (a bit more complicated example):

In [32]:
x_np = np.array([1, 2, 3, 4, 5])
x = tf.Variable(x_np, name="x", dtype=tf.float32)

with tf.GradientTape(persistent=True, watch_accessed_variables=True) as tape:
    # watch_accessed_variables=True allows us to not have to set each variable we want to watch
    
    z = tf.constant(7, dtype=tf.float32)
    #z = tf.Variable([7, 7, 7, 7, 7], dtype=tf.float32, name='z')
    
    y = z * tf.math.log(x)
    
print(y)

tf.Tensor([ 0.         4.8520303  7.690286   9.704061  11.266066 ], shape=(5,), dtype=float32)


In [33]:
grad_y_wrt_x = tape.gradient(target=y, sources=x)
print(grad_y_wrt_x)

tf.Tensor([7.        3.5       2.3333335 1.75      1.4      ], shape=(5,), dtype=float32)


In [34]:
grad_y_wrt_x2 = tape.gradient(target=y, sources=x) # we can grab it again

In [35]:
# we can even grab the gradient with respect to something else (e.g., z)
grad_y_wrt_z = tape.gradient(target=y, sources=z)
print(grad_y_wrt_z)

None


## So this is nice but what can we do with it? Let's train linear regression model with Tensorflow!

### For this, we'll create some simple data

In [36]:
# First make up a model
true_theta = tf.constant([-1, 5, 2, -7, 3], dtype=tf.float32)[:, tf.newaxis]
true_theta

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-1.],
       [ 5.],
       [ 2.],
       [-7.],
       [ 3.]], dtype=float32)>

In [37]:
n = 1500
ntr = 1000

# make some random data
x = tf.constant(tf.random.uniform((n, 5), minval=-1, maxval=+1), dtype=tf.float32)

# now calculate the y based on the true parameters
y = tf.constant(x @ true_theta, dtype=tf.float32)

# split the data
train_x = x[:ntr,:]
train_y = y[:ntr]

val_x = x[ntr:,:].numpy()
val_y = y[ntr:].numpy()

In [38]:
# This is batch gradient descent
def train_lr_tf(x, y, eta=0.05, num_iter=250, verbose=False):
    
    n, m = x.shape
    
    # weights / parameters (randomly initialized)
    theta = tf.Variable(tf.random.uniform((m, 1), minval=-1, maxval=1), dtype=tf.float32)
        
    for i in range(0, num_iter):
        
        with tf.GradientTape() as tape:
            y_pred = tf.linalg.matmul(x, theta) # prediction
            mse = tf.reduce_mean(tf.square(y - y_pred)) 
        
        # extract the gradients 
        gradient_vec = tape.gradient(mse, theta)

        # do a gradient descent step (we use assign_sub() to update theta in place)
        theta.assign_sub(tf.constant([eta], dtype=tf.float32) * gradient_vec) 


        if verbose and i % int(num_iter/10) == 0:
            print('Iteration {}: the (training) loss (MSE) is {:.5f}'.format(i, mse))
    
    return theta

In [39]:
# Let's do the training
theta = train_lr_tf(x, y, verbose=True)

Iteration 0: the (training) loss (MSE) is 28.77184
Iteration 25: the (training) loss (MSE) is 5.35863
Iteration 50: the (training) loss (MSE) is 1.00340
Iteration 75: the (training) loss (MSE) is 0.18878
Iteration 100: the (training) loss (MSE) is 0.03567
Iteration 125: the (training) loss (MSE) is 0.00676
Iteration 150: the (training) loss (MSE) is 0.00129
Iteration 175: the (training) loss (MSE) is 0.00025
Iteration 200: the (training) loss (MSE) is 0.00005
Iteration 225: the (training) loss (MSE) is 0.00001


In [40]:
print(theta)

<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[-0.9995907],
       [ 4.998418 ],
       [ 1.9997115],
       [-6.998523 ],
       [ 2.9993668]], dtype=float32)>


In [41]:
# given model parameters 'theta' and a feature matrix 'x', this will return predictions
def predict_theta(theta, x):
    return np.dot(x, theta) # note: there is no bias 'b' in this case
    
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error

def print_scores(desc, true_y, pred_y):
    r2 = r2_score(true_y, pred_y)
    rmse = mean_squared_error(true_y, pred_y, squared=False)
    medae = median_absolute_error(true_y, pred_y)
    
    print('[{}] R^2: {:.2f}, RMSE: {:.2f}, MedAE: {:.2f}'.format(desc, r2, rmse, medae))
        
print_scores('TF-GD Train', train_y, predict_theta(theta.numpy(), train_x))
print_scores('TF-GD Val', val_y, predict_theta(theta.numpy(), val_x))

[TF-GD Train] R^2: 1.00, RMSE: 0.00, MedAE: 0.00
[TF-GD Val] R^2: 1.00, RMSE: 0.00, MedAE: 0.00


## This is nice but it seems tedious. Do we have to implement the gradient descent ourselves and do all the low-level stuff?
### => No, we can use a higher-level API like Keras.

In [42]:
# This is the function to define the architecture
def create_model(input_shape, num_outputs=1):
    
    model = keras.models.Sequential()
    
    # declare input layer (keras needs to know the number of input features to expect)
    model.add(keras.Input(shape=(input_shape[1],))) 
    
    # next add our output layer (1 output, linear activation function)
    model.add(keras.layers.Dense(num_outputs, activation='linear'))
    
    return model

In [43]:
# first we create the model (i.e., define the architecture)
model = create_model(train_x.shape)

# Tip: before you go on, use summary() to check that the architecture is what you intended
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [44]:
# then we compile it to specify optimizer, loss, and metrics
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])

In [45]:
# finally, we train the model
model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(val_x, val_y))

Epoch 1/100
20/20 [==============================] - 0s 6ms/step - loss: 26.3165 - mae: 4.1876 - val_loss: 25.0808 - val_mae: 4.1231
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 20.3852 - mae: 3.6921 - val_loss: 19.4303 - val_mae: 3.6274
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 15.7941 - mae: 3.2478 - val_loss: 15.0541 - val_mae: 3.1917
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 12.2403 - mae: 2.8572 - val_loss: 11.6641 - val_mae: 2.8084
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 9.4860 - mae: 2.5180 - val_loss: 9.0377 - val_mae: 2.4714
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 7.3543 - mae: 2.2166 - val_loss: 7.0027 - val_mae: 2.1747
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 5.6970 - mae: 1.9514 - val_loss: 5.4266 - val_mae: 1.9139
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 4.

Epoch 62/100
20/20 [==============================] - 0s 2ms/step - loss: 4.8957e-06 - mae: 0.0018 - val_loss: 4.6223e-06 - val_mae: 0.0018
Epoch 63/100
20/20 [==============================] - 0s 1ms/step - loss: 3.8002e-06 - mae: 0.0016 - val_loss: 3.5877e-06 - val_mae: 0.0016
Epoch 64/100
20/20 [==============================] - 0s 2ms/step - loss: 2.9493e-06 - mae: 0.0014 - val_loss: 2.7845e-06 - val_mae: 0.0014
Epoch 65/100
20/20 [==============================] - 0s 1ms/step - loss: 2.2896e-06 - mae: 0.0012 - val_loss: 2.1602e-06 - val_mae: 0.0012
Epoch 66/100
20/20 [==============================] - 0s 2ms/step - loss: 1.7773e-06 - mae: 0.0011 - val_loss: 1.6769e-06 - val_mae: 0.0011
Epoch 67/100
20/20 [==============================] - 0s 2ms/step - loss: 1.3793e-06 - mae: 9.5352e-04 - val_loss: 1.3009e-06 - val_mae: 9.3623e-04
Epoch 68/100
20/20 [==============================] - 0s 2ms/step - loss: 1.0705e-06 - mae: 8.4032e-04 - val_loss: 1.0106e-06 - val_mae: 8.2515e-04
Epoc

In [46]:
# can we extract the parameters?
def extract_weights(model):
    for layer in model.layers:
        return layer.get_weights()

### What are the weights? Are they similar as before?

In [47]:
weights = extract_weights(model)
print(weights)

[array([[-0.9999956],
       [ 4.999971 ],
       [ 1.9999985],
       [-6.999973 ],
       [ 2.9999857]], dtype=float32), array([2.5642566e-07], dtype=float32)]


## Let's try a more complex problem with a more complex neural network architecture

### We'll use the Adult data

In [48]:
### In this case, we'll directly load the Adult dataset pre-processed in a similar way as for assignment 1
### and we'll immediately split it into train, test, validation.

train_x, train_y, test_x, test_y, val_x, val_y, features, labels = utils.load_preproc_adult(prop_vec=prop_vec, seed=seed)

# check that we have what we expect
print('Training: {}, {}'.format(train_x.shape, train_y.shape))
print('Test: {}, {}'.format(test_x.shape, test_y.shape))
print('Validation: {}, {}'.format(val_x.shape, val_y.shape))

Training: (38762, 88), (38762,)
Test: (3231, 88), (3231,)
Validation: (3229, 88), (3229,)


### In assignment 2 we had found the best model was a SVM classifier which achieved around 85% accuracy. Can we do better?

In [49]:
# This is the function to define the architecture
def create_model_adult(input_shape, hidden_widths=[96, 32], num_outputs=1):
    
    model = keras.models.Sequential()
    
    # declare input layer (keras needs to know the number of input features to expect)
    model.add(keras.Input(shape=(input_shape[1],))) 
    
    # add two hidden layers with ReLU activation
    model.add(keras.layers.Dense(hidden_widths[0], activation='relu'))
    model.add(keras.layers.Dense(hidden_widths[1], activation='relu'))
    
    # next add our output layer (binary classification with 1 output, so sigmoid makes the most sense)
    model.add(keras.layers.Dense(num_outputs, activation='sigmoid'))
    
    return model

In [50]:
# create the model (i.e., define the architecture)
model = create_model_adult(train_x.shape)

# Tip: before you go on, use summary() to check that the architecture is what you intended
model.summary()

# then we compile it to specify optimizer, loss, and metrics
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 96)                8544      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                3104      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 11,681
Trainable params: 11,681
Non-trainable params: 0
_________________________________________________________________


In [51]:
# we train the model
model.fit(x=train_x, y=train_y, epochs=100, batch_size=100, validation_data=(val_x, val_y))

Epoch 1/100
388/388 [==============================] - 1s 2ms/step - loss: 0.4807 - accuracy: 0.7734 - val_loss: 0.3913 - val_accuracy: 0.8210
Epoch 2/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3657 - accuracy: 0.8299 - val_loss: 0.3540 - val_accuracy: 0.8346
Epoch 3/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3456 - accuracy: 0.8372 - val_loss: 0.3435 - val_accuracy: 0.8374
Epoch 4/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3366 - accuracy: 0.8427 - val_loss: 0.3377 - val_accuracy: 0.8421
Epoch 5/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3308 - accuracy: 0.8462 - val_loss: 0.3332 - val_accuracy: 0.8421
Epoch 6/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3268 - accuracy: 0.8483 - val_loss: 0.3300 - val_accuracy: 0.8421
Epoch 7/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3239 - accuracy: 0.8497 - val_loss: 0.3274 - val_accuracy: 0.8442

Epoch 58/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3077 - accuracy: 0.8563 - val_loss: 0.3161 - val_accuracy: 0.8467
Epoch 59/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3077 - accuracy: 0.8569 - val_loss: 0.3153 - val_accuracy: 0.8492
Epoch 60/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3077 - accuracy: 0.8573 - val_loss: 0.3150 - val_accuracy: 0.8507
Epoch 61/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3075 - accuracy: 0.8564 - val_loss: 0.3155 - val_accuracy: 0.8483
Epoch 62/100
388/388 [==============================] - 1s 1ms/step - loss: 0.3074 - accuracy: 0.8561 - val_loss: 0.3153 - val_accuracy: 0.8476
Epoch 63/100
388/388 [==============================] - 1s 1ms/step - loss: 0.3073 - accuracy: 0.8570 - val_loss: 0.3167 - val_accuracy: 0.8461
Epoch 64/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3071 - accuracy: 0.8571 - val_loss: 0.3151 - val_accuracy:

In [52]:
loss, accuracy = model.evaluate(x=test_x, y=test_y, verbose=0)
print('Test accuracy: {:.2f}%'.format(accuracy*100))

Test accuracy: 85.52%


## Let's use TensorBoard

In [53]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [54]:
model = create_model_adult(train_x.shape)
#model.summary()
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# set up tensorboard log directory and callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=train_x, y=train_y, epochs=100, batch_size=100, validation_data=(val_x, val_y), 
          callbacks=[tensorboard_callback])

Epoch 1/100
  1/388 [..............................] - ETA: 0s - loss: 0.6455 - accuracy: 0.7900WARNING:tensorflow:From /home/tony/anaconda3/envs/deep-learning-3.8/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
388/388 [==============================] - 1s 2ms/step - loss: 0.4544 - accuracy: 0.7929 - val_loss: 0.3814 - val_accuracy: 0.8309
Epoch 2/100
388/388 [==============================] - 1s 1ms/step - loss: 0.3589 - accuracy: 0.8346 - val_loss: 0.3541 - val_accuracy: 0.8340
Epoch 3/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3432 - accuracy: 0.8400 - val_loss: 0.3450 - val_accuracy: 0.8368
Epoch 4/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3356 - accuracy: 0.8429 - val_loss: 0.3403 - val_accuracy: 0.8417
Epoch 5/100
388/388 [============

Epoch 54/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3096 - accuracy: 0.8546 - val_loss: 0.3158 - val_accuracy: 0.8467
Epoch 55/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3094 - accuracy: 0.8551 - val_loss: 0.3161 - val_accuracy: 0.8495
Epoch 56/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3094 - accuracy: 0.8547 - val_loss: 0.3163 - val_accuracy: 0.8470
Epoch 57/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3092 - accuracy: 0.8550 - val_loss: 0.3161 - val_accuracy: 0.8483
Epoch 58/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3090 - accuracy: 0.8558 - val_loss: 0.3166 - val_accuracy: 0.8470
Epoch 59/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3090 - accuracy: 0.8553 - val_loss: 0.3157 - val_accuracy: 0.8501
Epoch 60/100
388/388 [==============================] - 1s 2ms/step - loss: 0.3090 - accuracy: 0.8554 - val_loss: 0.3154 - val_accuracy:

In [57]:
# Start tensorboard (notebook experience)
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 14903), started 0:47:30 ago. (Use '!kill 14903' to kill it.)